# Chapter 8

In [3]:
%%file chapter8_blocks.sp

#maxint=1000.
sorts
#block=[b][0..7].
#location=#block+{t}.
#inertial_fluent=on(#block(X),#location(Y)):X!=Y.
#defined_fluent=above(#block(X),#location(Y)):X!=Y.
#fluent=#inertial_fluent + #defined_fluent.
#action=put(#block(X),#location(Y)):X!=Y.
#step=0..2.
predicates
holds(#fluent,#step).
occurs(#action,#step).

rules
%initial:
holds(on(b0,t),0).
holds(on(b3,b0),0).
holds(on(b2,b3),0).
holds(on(b1,t),0).
holds(on(b4,b1),0).
holds(on(b5,t),0).
holds(on(b6,b5),0).
holds(on(b7,b6),0).
%% If block B is not known to be on location L at step 0,
%% then we assume it is not.
-holds(on(B,L),0) :- not holds(on(B,L),0).
holds(on(B,L),I+1) :- occurs(put(B,L),I).
%rule1
-holds(on(B,L2),I) :- holds(on(B,L1),I),L1 != L2.
%% rule 2
%Another useful rule says that no block can support more than one block
%directly on top of it
-holds(on(B2,B),I) :- holds(on(B1,B),I),B1 != B2,#block(B).
%inertia axioms:

holds(F,I+1) :- #inertial_fluent(F), holds(F,I), not -holds(F,I+1).
 
-holds(F,I+1) :- #inertial_fluent(F), -holds(F,I), not holds(F,I+1). 
    
%%rule3 - defining blocks above other blocks

holds(above(B,L),I) :- holds(on(B,L),I).
holds(above(B,L),I) :- holds(on(B,B1),I), holds(above(B1 ,L),I).
-holds(above(B,L),I) :-  #defined_fluent(F), not holds(above(B,L),I).

%impossibility:
-occurs(put(B,L),I) :- holds(on(B1,B),I). %% rule 5a
-occurs(put(B1,B),I) :- holds(on(B2,B),I), #block(B). %% rule 5b

%some actions:
occurs(put(b2,t),0).
occurs(put(b7,b2),1).

Overwriting chapter8_blocks.sp


In [ ]:
! java -jar sparc.jar chapter8_blocks.sp

SPARC  V2.57
program translated
?- 

In [4]:
%%file briefcase.lp

%%Domain Signature
clasp(1).
clasp(2).

fluent( inertial , up(C)) :- clasp(C).
fluent( defined , open).
action(toggle(C)) :- clasp(C).
#const n = 1.
step(0..n).

%% toggle(C) causes up(C) if -up(C)
holds(up(C),I+1) :- occurs(toggle(C),I),
-holds(up(C), I),
I < n.

%% toggle(C) causes -up(C) if up(C)
-holds(up(C),I+1) :- occurs(toggle(C),I),
holds(up(C),I),
I < n.

%% open if up(1), up(2).
holds(open ,I) :- holds(up(1),I),
holds(up(2),I).

%% CWA for Defined Fluents
-holds(F,I) :- fluent( defined ,F),
step(I),
not holds(F,I).


%% General Inertia Axiom
holds(F,I+1) :- fluent( inertial ,F),
holds(F,I),
not -holds(F,I+1),
I < n.
-holds(F,I+1) :- fluent (inertial ,F),
-holds(F,I),
not holds(F,I+1),
I < n.

%% CWA for actions
-occurs(A,I) :- action(A), step(I),
not occurs(A,I).

%% Initial Situation
-holds(up(1),0).
holds(up(2),0).
-holds(open ,0).
%% Action
occurs(toggle (1),0).
%% Display
#show holds/2.
#show -holds/2.

Overwriting briefcase.lp


In [5]:
! clingo briefcase.lp

clingo version 5.7.1
Reading from briefcase.lp
Solving...
Answer: 1
holds(up(2),0) holds(up(1),1) -holds(up(1),0) -holds(open,0) holds(open,1) holds(up(2),1)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


In [10]:
%%file chapter8_blocksal.lp

block(b0). block(b1). block(b2). block(b3).
block(b4). block(b5). block(b6). block(b7).

location(X) :- block(X).
location(t).

fluent( inertial , on(B,L)) :- block(B), location(L).
fluent( defined , above (B,L)) :- block(B), location(L).
action(put(B,L)) :- block(B), location(L),
B != L.

#const n = 1.
step(0..n).

holds(on(B,L),I+1) :- occurs(put(B,L),I), I < n.
    
-holds(on(B,L2),I) :- holds(on(B,L1),I), location(L2),L1 != L2.
    
-holds(on(B2 ,B),I) :- holds(on(B1 ,B),I),block (B), 
    block(B2),
    B1 != B2.
    
holds(above(B2 ,B1),I) :- holds(on(B2 ,B1),I).

holds(above(B2 ,B1),I) :- holds(on(B2 ,B),I),
holds(above(B,B1),I).

-occurs(put(B,L),I) :- location (L),
holds(on(B1 ,B),I).

-occurs(put(B1 ,B),I) :- block(B1), block(B),
holds(on(B2 ,B),I).

%% CWA for Defined Fluents

-holds(F,I) :- fluent( defined ,F), step (I), not holds(F,I).
    
%% General Inertia Axiom

holds(F,I+1) :- fluent( inertial ,F),
holds(F,I),
not -holds(F,I+1),
I < n.

-holds(F,I+1) :- fluent (inertial ,F),
-holds(F,I),
not holds(F,I+1),
I < n.

%% CWA for Actions
-occurs(A,I) :- action(A), step(I),
not occurs(A,I).

-occurs(put(B1 ,L),I) | -occurs(put(B2 ,B1),I) :- step(I), action(put(B1 ,L)),action(put(B2 ,B1)).

occurs(put(b2 ,t) ,0).
occurs(put(b4 ,b7) ,0).

Overwriting chapter8_blocksal.lp


In [11]:
! clingo chapter8_blocksal.lp

clingo version 5.7.1
Reading from chapter8_blocksal.lp
Solving...
Answer: 1
block(b0) block(b1) block(b2) block(b3) block(b4) block(b5) block(b6) block(b7) location(t) location(b0) location(b1) location(b2) location(b3) location(b4) location(b5) location(b6) location(b7) fluent(inertial,on(b0,t)) fluent(inertial,on(b0,b0)) fluent(inertial,on(b0,b1)) fluent(inertial,on(b0,b2)) fluent(inertial,on(b0,b3)) fluent(inertial,on(b0,b4)) fluent(inertial,on(b0,b5)) fluent(inertial,on(b0,b6)) fluent(inertial,on(b0,b7)) fluent(inertial,on(b1,t)) fluent(inertial,on(b1,b0)) fluent(inertial,on(b1,b1)) fluent(inertial,on(b1,b2)) fluent(inertial,on(b1,b3)) fluent(inertial,on(b1,b4)) fluent(inertial,on(b1,b5)) fluent(inertial,on(b1,b6)) fluent(inertial,on(b1,b7)) fluent(inertial,on(b2,t)) fluent(inertial,on(b2,b0)) fluent(inertial,on(b2,b1)) fluent(inertial,on(b2,b2)) fluent(inertial,on(b2,b3)) fluent(inertial,on(b2,b4)) fluent(inertial,on(b2,b5)) fluent(inertial,on(b2,b6)) fluent(inertial,on(b2,b7)) fl

# Question 1

Prove that
(a) Every state of system description SD satisfies the state constraints
of SD.

State in SD is a consistent set of fluent literals. State constraints are imposed on these fluent literals and must always be satisfied by them; any action a leading to a new state will have to lead to a state, by the inertia and transition laws, that is also satisfied by the state constrains.

(b) If the signature of SD does not contain defined fluents, a state is a
complete, consistent set of literals satisfying the state constraints
of SD.

Fluents are divied into intertial and defined fluents, where inertial fluents can be changed by actions and are subject to the inertia axiom (will stay the same unless acted upon) while defined fluents are not subject to the inertia axiom and are defined by other fluents. A state of SD is complete if it contains p or -p for any domain property p, and consistent if there is no domain property that has both p and -p within SD. Without defined fluents, as given by part a, any action will have to lead to a state that still satisfies the state constraints, as all domain properties are subject to the inertia axiom.

# Question 2

Given the following AL system description where fluents f and g are
inertial and h is defined,

*a **causes** f **if** g*
*h **if** , g*

(a) Show its translation into the corresponding ASP program.

holds(f, I+1) :- holds(g, I),
occurs(a, I),
I<n. 

holds(h, I) :- holds(g, I).


(b) Check if each of the following is a valid state:

i. σ = {f, ¬g, ¬h} - yes
ii. σ = {¬f, ¬g, ¬h} - yes
iii. σ = {¬f, g, h} - no
iv. σ = {f, g, ¬h} - no


(c) Draw the transition diagram for the system.


# Question 3

Consider the system description presented in Section 8.6.

(a) Explain why {f, p, q} is not a valid state of this system description.

Non-deterministic state - by rules 2 and 3, we should either have -q or -p.

(b) Add causal law b causes ¬f if f to the system description and
draw the corresponding transition diagram

1. a causes f if ¬f
2. ¬q if f, p
3. ¬p if f, q
3. b causes ¬f if f

(same as before but b with arrows going back where a is going?)

# Question 4

4. Given the following story: Jenny painted the wall white.

(a) Represent the story in AL. Assume that to paint a wall a given
color, one must have paint of the appropriate color. Initially the
wall is yellow and Jenny has the white paint. Jenny paints the wall
at step 0. Make sure that your theory entails that at the end of the
story the wall is white and not yellow.

jenny causes white_wall if has_white_paint.


(b) Translate the representation to ASP and run it using an ASP solver
to predict the values of fluents if Jenny paints the wall white.




(c) Now suppose Jenny has black paint as well and, after painting the
wall white, decides to paint it black. Use ASP to do some temporal
projection about the values of the fluents after both actions are
performed sequentially.



In [1]:
%%file chapter8_question4_paintingwalls.sp

#maxint=1000.
sorts
#inertial_fluent={white_wall} + {yellow_wall} + {black_wall}.
#defined_fluent={has_white} + {has_black}.
#fluent=#inertial_fluent + #defined_fluent.
#action={paint_wall_white} + {paint_wall_black}.
#step=0..2.
predicates
holds(#fluent,#step).
occurs(#action,#step).

rules
%initial:
holds(yellow_wall,0).
holds(has_white,0).
holds(has_black,0).
%% 
-holds(has_white,0) :- not holds(has_white,0).
holds(has_white,I+1) :- holds(has_white,I).
-holds(has_black,0) :- not holds(has_black,0).
holds(has_black,I+1) :- holds(has_black,I).

%inertia axioms:

holds(F,I+1) :- #inertial_fluent(F), holds(F,I), not -holds(F,I+1).
 
-holds(F,I+1) :- #inertial_fluent(F), -holds(F,I), not holds(F,I+1). 
    
%%rule
    
holds(white_wall,I+1) :- holds(has_white, I), 
    occurs(paint_wall_white,I),
    I<n.
    
holds(black_wall,I+1) :- holds(has_black, I), 
    occurs(paint_wall_black,I),
    I<n.

%some actions:
occurs(paint_wall_white,0).
occurs(paint_wall_black,1).
    
%now outputs yes to query holds(white_wall, 1).
%could definitely be coded better next time where we have colours in some sort of set 

Overwriting chapter8_question4_paintingwalls.sp


In [ ]:
! java -jar sparc.jar chapter8_question4_paintingwalls.sp

SPARC  V2.57
program translated
?- 

# Question 5

Given the following story: Claire always carries her cell phone with
her. Claire is at the library.

(a) Represent the story in AL. Include any commonsense knowledge
necessary to answer the question, “Where is Claire’s cell phone?”
Make the representation general enough so that when you add
the fact that Claire went home, her cell phone’s location changed
accordingly. Also make it general enough that if we change the cell
phone to a pet chihuahua, your program will still make the proper
conclusions. Assume locations are distinct. Hint: Use the inertial
fluent carried(Obj, P erson) – the object is carried by the person.

statics = {claire, objects}
cell phone subclass of objects
inertial_fluents = carried(object, person)
defined_fluents = at(object, location), at(person,location)
actions = go(location, person)

state constraints:

if carried(object,person) then at(object,location) :- at(person, location)

(otherwise inertia axiom)

causal law:

if go(location, person) :- at(person, location)

(b) Translate the representation to ASP and run it using an ASP solver
to predict the values of fluents after Claire returns home.

(c) Modify your program to include that Rod carries a towel with him
everywhere and that Claire and Rod are never at the same place
at the same time. Make sure that your program can conclude that
Claire’s cell phone and Rod’s towel are also not at the same place at
the same time, even when it does not know where Rod is initially.

In [19]:
%%file chapter8_question5_cellphonesnchihuahas.sp

#maxint=1000.
sorts
#object = {cellphone,chihuaha, towel}.
#person = {claire, rod}.
#location = {home, library}.
#fluent=at(#object, #location) + carried(#object, #person) + at(#person, #location).
#action=go(#person, #location).
#step=0..2.
predicates
holds(#fluent,#step).
occurs(#action,#step).
    
rules
%initial:
holds(at(claire, library),0).
holds(carried(cellphone, claire),0).
holds(carried(towel, rod),0).
    
%inertia axioms:

holds(F,I+1) :- #fluent(F), holds(F,I), not -holds(F,I+1).
 
-holds(F,I+1) :- #fluent(F), -holds(F,I), not holds(F,I+1). 
       
%moving locations causes person to be at new location
       
holds(at(P, L), I+1) :- occurs(go(P, L), I), I < n.
    
-holds(at(P,L), I) :- holds(at(P,L2), I), 
    L != L2.
    
-holds(at(claire, L),I) :- holds(at(rod, L), I).

-holds(at(rod, L),I) :- holds(at(claire, L), I).

holds(at(O, L), I) :- holds(carried(O, P), I), holds(at(P, L), I), I < n.
    
-holds(at(O, L), I) :- not holds(at(O, L), I), I < n.
       
%what happens:   
occurs(go(claire, home),1).
    
%correctly outputs unknown to queries about towel at time 0 but no to if its at the lib since thats where claire is

Overwriting chapter8_question5_cellphonesnchihuahas.sp


# Question 7

Given the following story: Jonathan has requirements for playing the
Wii: He should make sure that his homework is done, the bed is made,
and he has practiced Tae Kwon Do. He can only do one thing at a
time. Of course, he cannot make the bed if it already made or do his
homework if it is already done or if none was assigned.
(a) Select an initial situation and a sequence of Jonathan’s actions that
would allow him to play the Wii. Represent the resulting story in
AL. Hint: Create a sort activity for homework, make bed, TKD,
and Wii. Then use actions do(hw), do(make bed), do(tkd), and
do(wii) to make it easy to express the requirement that actions be
mutually exclusive.
(b) Translate the representation to ASP and run it using an ASP solver
to answer questions about whether a boy may play the Wii at various
future moments. Make sure that the system description part of your
program works for other variants of this story